In [31]:
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
import branca
from branca.colormap import linear
from branca.element import MacroElement
from jinja2 import Template
import json
import base64
from folium import plugins

In [18]:
dfmain = pd.read_csv('data.csv')

For displaying multiple colormaps

In [32]:
class BindColormap(MacroElement):
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)

In [ ]:
m = folium.Map(
    location=[0, 0],
    zoom_start=3)
fgdict = {}

def geojson(LH, file, df):
  gdata = json.load(open(file))
  df = df[df["Frequency"] == "Monthly"][df["LocationHierarchy"] == LH]
  df = df[['Location', 'venezuelans', 'ven/pop', 'KML']]
  fgdict[LH] = folium.map.FeatureGroup(name=LH, show=False)

  colormap = linear.BuPu_09.scale(
      df.venezuelans.min(),
      df.venezuelans.max())
  colormap.caption = 'Absolute Measure'
  colormap2 = linear.YlGn_09.scale(
      df['ven/pop'].min(),
      df['ven/pop'].max())
  colormap2.caption = 'Relative Measure'

  m.add_child(colormap)
  m.add_child(colormap2)
  m.add_child(BindColormap(fgdict[LH], colormap)).add_child(BindColormap(fgdict[LH], colormap2))

  vdict = df.set_index('Location')['venezuelans']
  vdict2 = df.set_index('Location')['ven/pop']
  folium.GeoJson(gdata,
           style_function=lambda feature: {
          'fillColor': colormap(vdict[feature['properties']['name']]) if feature['properties']['name'] in vdict else 'grey',
          'color': colormap2(vdict2[feature['properties']['name']]) if feature['properties']['name'] in vdict2 else 'grey',
          'weight': 2,
          'fillOpacity': 1 if feature['properties']['name'] in vdict else 0} ).add_to(fgdict[LH])
  for feature in gdata['features']:
    fig1, ax1 = plt.subplots(figsize=(2.4,2.4))
    a = dfmain[dfmain["Location"] == feature['properties']['name']][dfmain["Frequency"] == "Monthly"]
    ax1.pie([a.loc[(a.index)[0],"%ven_audience_man"],a.loc[(a.index)[0],"%ven_audience_woman"]], labels=("Men", "Women"), autopct='%1.1f%%', shadow=True, startangle=90)
    plt.savefig('myfig.png', transparent = True)
    fig1, ax1 = plt.subplots(figsize=(2.4,2.4))
    ax1.pie([a.loc[(a.index)[0],"%ven_audience_iOS"],a.loc[(a.index)[0],"%ven_audience_Android"],a.loc[(a.index)[0],"%ven_audience_Other"]], labels=("iOS", "Android", "Others"), autopct='%1.1f%%', shadow=True, startangle=90)
    plt.savefig('myfig2.png', transparent = True)
    geo = folium.GeoJson(feature['geometry'],
           style_function=lambda feature: {
           'weight': 0,
           'fillOpacity': 0},
            tooltip =feature['properties']['name'])
    encoded = base64.b64encode(open('myfig.png', 'rb').read()).decode()
    encoded2 = base64.b64encode(open('myfig2.png', 'rb').read()).decode()
    html = '<center><h4>' + feature['properties']['name'] + '</h4><h5></h5></center><center><img align="middle" src="data:image/png;base64,{}"><img align="middle" src="data:image/png;base64,{}"></center>'
    iframe = branca.element.IFrame(html=html.format(encoded,encoded2), width=400, height=200)
    folium.Popup(iframe).add_to(geo)  
    geo.add_to(fgdict[LH])
  fgdict[LH].add_to(m)
  
geojson("City", 'geodata/cities.geojson', dfmain)
geojson("Country", 'geodata/countries.geojson', dfmain)
geojson("State", 'geodata/states.geojson', dfmain)
fgdict["None"] = folium.map.FeatureGroup(name="Non", show=False)
fgdict["None"].add_to(m)

#facts

def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

fgfacts = {}
def facts(LH, dfm):
    fgfacts[LH] = folium.map.FeatureGroup(name='Facts for ' + LH, show = False)
    Loc, tdict, pdict, pidict, hdict, cdict = [], {}, {}, {}, {}, {}
    df = dfm
    df = df[df['Frequency']=='Monthly'][df['LocationHierarchy']==LH]
    def cfacts(array, labels, icon):
      for i in range(len(array)):
        arr = [0]*len(array)
        a = df.loc[df[array[i]].idxmax(axis=1)]
        for j in range(len(array)):
            arr[j]=a[array[j]]
        fig1, ax1 = plt.subplots(figsize=(1.8,1.8))
        ax1.pie(arr, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
        plt.savefig('myfig.png', transparent = True)
        html1 = '<center><img align="middle" src="data:image/png;base64,{}"></center>'
        encoded = base64.b64encode(open(icon, 'rb').read()).decode()
        encpie = base64.b64encode(open('myfig.png', 'rb').read()).decode()
        ic = 'venflag2.png'
        l = a.Location
        if l not in Loc:
          Loc.append(l)
          pdict[l], hdict[l],tdict[l], pidict[l], cdict[l] = '<h4><center>' + l +'</center></h4>', '', [], [], 0
        pdict[l] += '<hr><center><p style="padding:0px 10px 0px 10px">'+ l
        hdict[l] += html1
        tdict[l].append(encoded)
        pidict[l].append(encpie)
        cdict[l]+=1
        string = " highest percentage of "+ labels[i]+"."
        pdict[l] += ' has the'+string+'</p></center><center><img align="middle" src="data:image/png;base64,{}"></center>' 
        iframe = branca.element.IFrame(html=pdict[l].format(*pidict[l]), width=400, height = 220 + cdict[l]* 50)
        folium.Marker([a.LatLong.split(",")[0], a.LatLong.split(",")[1]], popup = folium.Popup(iframe), icon = folium.features.CustomIcon(ic,icon_size=(28, 30)),tooltip=hdict[l].format(*tdict[l]) ).add_to(fgfacts[LH])

    cfacts(['%ven_audience_woman', '%ven_audience_man'], ('Women', 'Men'), 'iconr1.png')
    cfacts(['%ven_audience_iOS', '%ven_audience_Android', '%ven_audience_Other'], ('iOS', 'Android', 'Other'), 'iconr2.png')
    cfacts(['%ven_audience_graduated', '%ven_audience_high_school','%ven_audience_no_degree'], ('Graduated', 'High School', 'No degree'), 'iconr3.png')
    cfacts(['%ven_audience_single', '%ven_audience_dating', '%ven_audience_married'], ('Single', 'Dating', 'Married'), 'iconr4.png')
    fgfacts[LH].add_to(m)

facts("City", dfmain)
facts("Country", dfmain)
facts("State", dfmain)
fgfacts["None"] = folium.map.FeatureGroup(name="Non", show = False)
fgfacts["None"].add_to(m)

fgsparkline = {}
def sparkline(LH, dfm):
    fgsparkline[LH] = folium.map.FeatureGroup(name=LH, show = False)
    df = dfm[dfm["Frequency"] == "Monthly"][dfm["LocationHierarchy"] == LH]
    df = df[~df["KML"].isnull()]
    plotdf = pd.DataFrame([df.Location]).transpose()
    li = list(dfm)
    str = "venezuelans"
    for i in range(len(list(dfm))):
      col = li[i]
      if len(col) == 16:
        if (col[0:11] == str) & (col[13] == '_'):
          plotdf[col]=dfm[col]
    cols = plotdf.columns
    plotindex = plotdf.index

    for i in plotindex:
      x = plotdf[plotdf["Location"] == plotdf.loc[i, "Location"]]
      x=x.T.convert_objects(convert_numeric=True)
      x.plot(legend=False, color="black", figsize = (3 , 0.5))
      plt.axis('off')
      plt.savefig('mark.png', transparent = True)
      icon = folium.features.CustomIcon('mark.png',icon_size=(100, 20))
      # tooltip=df1.loc[ind[n], "Location"]
      folium.Marker([dfmain.loc[i, "LatLong"].split(",")[0], 
                     dfmain.loc[i, "LatLong"].split(",")[1]], 
                    tooltip="Sparkline representing Venezuelan population", 
                    icon=icon ).add_to(fgsparkline[LH])
    fgsparkline[LH].add_to(m)
sparkline("City", dfmain)
sparkline("Country", dfmain)
sparkline("State",dfmain)
fgsparkline["None"] = folium.map.FeatureGroup(name="None", show = False)
fgsparkline["None"].add_to(m)


folium.plugins.GroupedLayerControl({}, 
                                    {'Location Hierarchy' : {'Cities' : fgdict["City"], 'Countries' : fgdict["Country"], 'States' : fgdict["State"], "None " : fgdict["None"]},
                                    'Facts': {"Cities" : fgfacts["City"], "Countries" : fgfacts["Country"], "State" : fgfacts["State"], "None " : fgfacts["None"]}, 
                                    'Sparkline': {"Cities" : fgsparkline["City"], "Countries" : fgsparkline["Country"], "State" : fgsparkline["State"], "None" : fgsparkline["None"]}}, 
                                  ['Location Hierarchy', 'Facts', 'Sparkline'], [fgdict["City"]]).add_to(m)


In [36]:
m.save('June2.html')

61 MB output file size

#### Changes to be made

1. Simplify kml
2. ~~Include code for multiple colormaps~~
3. Change signature of function ~~include df~~
4. More info to information box
5. Default state of plugin
6. Customised colormaps
7. Geojson - tooltip and popup

Issues -
1. Geojson tooltip - countries, images
2. Place names in simplified KML files